In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# Path to the folder containing the Excel files
folder_path = "C:\\Users\\natha\\OneDrive\\Documents\\Marijuana\\Marijuana-Legalization-CDL\\BLS Excels"

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Create a dictionary to store DataFrames
dataframes = {}

# Loop through the files
for file_name in file_list:
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):  # Check if the file is an Excel file
        # Extract year from the file name (necessary becuase the BLS is inconsistent with file naming)
        numbers = ''.join(filter(str.isdigit, file_name))
        
        #sheets from before the year 2000 do not have a field for the median wage across all occupations to use as a baseline
        try:
            int_num = int(numbers)
        except ValueError:
            continue
        
        #hardcoded exception due to BLS change in format from 2000 to 2001

        if int_num > 2000:
        # Create DataFrame
            df = pd.read_excel(os.path.join(folder_path, file_name))
        
        # Store DataFrame in the dictionary
            dataframes[numbers] = df





In [3]:
#import local text file containing all CDL codes
relevant_occ_codes = list(pd.read_csv("C:\\Users\\natha\\OneDrive\\Documents\\Marijuana\\Marijuana-Legalization-CDL\\Occupation Codes.txt"))

In [4]:
#standardize all column headers to lower case because the BLS loves changing their format more than a daytime gameshow
for year in dataframes.keys():
    df = dataframes[year]
    df.columns = df.columns.str.lower()
    dataframes[year] = df

In [5]:
#keep only relevant occ codes
for year in dataframes.keys():
    df = dataframes[year]
    df = df[df['occ_code'].isin(relevant_occ_codes)]
    df = df.reset_index()
    dataframes[year] = df

In [6]:
"""The BLS is extremely inconsistent with their column labels.
This cell will take every unique column name and write it to a CSV. 
I will then manually review that CSV, delete and irrelevant columns, 
and write a mapping to rename every column to allow us to do a proper
merge across all 21 excel sheets we pulled."""
#column_names_list = []
#for year in dataframes.keys():
#    df = dataframes[year]
#    for col_name in list(df.columns):
#        if not col_name in column_names_list:
#            column_names_list.append(col_name)
#column_names_list            

'The BLS is extremely inconsistent with their column labels.\nThis cell will take every unique column name and write it to a CSV. \nI will then manually review that CSV, delete and irrelevant columns, \nand write a mapping to rename every column to allow us to do a proper\nmerge across all 21 excel sheets we pulled.'

In [7]:
#add a year column to all sheets that didn't have one when released by BLS                                   
for year in dataframes.keys():
    df = dataframes[year]
    if "year" not in df.columns:
        df.insert(loc = len(df.columns), column = "year", value = year)

In [8]:
dataframes["2002"]

,index,area,st,state,occ_code,occ_title,group,tot_emp,emp_prse,h_mean,...,h_median,h_wpct75,h_wpct90,a_wpct10,a_wpct25,a_median,a_wpct75,a_wpct90,annual,year
0,0,1,AL,Alabama,00-0000,All Occupations,major,1819390,0.5,14.88,...,11.37,18.13,26.94,13250,16420,23640,37710,56040,NaN,2002
1,623,1,AL,Alabama,53-3021,"Bus drivers, transit and intercity",NaN,1680,29.9,9.32,...,7.91,11.04,14.59,11770,13040,16450,22960,30350,NaN,2002
2,624,1,AL,Alabama,53-3022,"Bus drivers, school",NaN,7300,9.7,6.55,...,6.43,6.93,7.78,11710,12350,13370,14410,16190,NaN,2002
3,626,1,AL,Alabama,53-3032,"Truck drivers, heavy and tractor-trailer",NaN,31530,2.7,15.44,...,14.63,19.57,23.09,18400,22920,30420,40700,48040,NaN,2002
4,627,1,AL,Alabama,53-3033,"Truck drivers, light or delivery services",NaN,15250,3.9,12.12,...,10.26,14.27,22.89,13740,16540,21350,29680,47610,NaN,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,32186,72,PR,Puerto Rico,53-3033,"Truck drivers, light or delivery services",NaN,6770,6.3,7.31,...,6.6,7.48,9.68,11740,12500,13720,15560,20140,NaN,2002
258,32204,78,VI,Virgin Islands,00-0000,All Occupations,major,40980,1.1,12.86,...,10.52,15.47,22.57,13230,16020,21870,32170,46950,NaN,2002
259,32391,78,VI,Virgin Islands,53-3022,"Bus drivers, school",NaN,90,0.8,11.08,...,10.52,12.97,16.16,16570,18240,21890,26980,33610,NaN,2002
260,32392,78,VI,Virgin Islands,53-3032,"Truck drivers, heavy and tractor-trailer",NaN,290,8.3,11.7,...,11.02,13.51,16.74,17340,19870,22920,28090,34810,NaN,2002


In [9]:
#remove all columns not necessary for our simple median wage analysis. Can be left out to do a more complex analysis at a later time. Also standardize state name
dataframes2 = {}
column_names = list(pd.read_csv("C:\\Users\\natha\\OneDrive\\Documents\\Marijuana\\Marijuana-Legalization-CDL\\Column Names.txt", header=None).iloc[0])
for year in dataframes.keys():
    df = dataframes[year]
    cols_in_this_df = [col_name for col_name in df.columns if col_name in column_names]
    df = df[cols_in_this_df]
    df = df.rename(mapper={"area_title":"state"}, axis=1)
    dataframes[year] = df

In [10]:
dataframes["2002"]

,state,occ_code,tot_emp,h_median,year
0,Alabama,00-0000,1819390,11.37,2002
1,Alabama,53-3021,1680,7.91,2002
2,Alabama,53-3022,7300,6.43,2002
3,Alabama,53-3032,31530,14.63,2002
4,Alabama,53-3033,15250,10.26,2002
...,...,...,...,...,...
257,Puerto Rico,53-3033,6770,6.6,2002
258,Virgin Islands,00-0000,40980,10.52,2002
259,Virgin Islands,53-3022,90,10.52,2002
260,Virgin Islands,53-3032,290,11.02,2002


In [11]:
#keep only the 50 states and DC, territories have significantly differnet and incomparable economies
territories_list = ["Virgin Islands", "Puerto Rico", "Guam"]
for year in dataframes.keys():
    df = dataframes[year]
    df = df[~df["state"].isin(territories_list)]
    dataframes[year] = df

In [12]:
#merge everything into one DF
years = [str(year) for year in range(2001, 2023)]
full_df = pd.concat(objs=dataframes.values())
full_df = full_df.reset_index().drop(labels="index", axis=1)
full_df

,state,occ_code,tot_emp,h_median,year
0,Alabama,00-0000,1827960,11.02,2001
1,Alabama,53-3021,1290,8.4,2001
2,Alabama,53-3022,8400,6.27,2001
3,Alabama,53-3032,33270,14.37,2001
4,Alabama,53-3033,14810,10.26,2001
...,...,...,...,...,...
5570,Wyoming,00-0000,283980,15.52,2008
5571,Wyoming,53-3021,210,11.08,2008
5572,Wyoming,53-3022,1130,11.11,2008
5573,Wyoming,53-3032,6270,17.60,2008


In [37]:
#replace occ_code with descriptor for driving and other for groupby later, also replace nans appropriately
replace_dict = {"00-0000" : "all"}
for code in relevant_occ_codes[1:]:
    replace_dict[code] = "driving"
full_df = full_df.replace(replace_dict)
nan_replace_dict = {
                    "*" : np.nan,
                    "**" : np.nan,
                    "***" : np.nan
                    }
full_df = full_df.replace(nan_replace_dict)

In [39]:
#sort full_df
full_df = full_df.sort_values(by=["year", "state"])

In [32]:
driving_only = full_df[full_df["occ_code"] == "driving"]

In [40]:
#combine driving 
combine_driving_test = driving_only.copy()


In [44]:
#apply function to numeric columns that have floats to cast all objects to float or else replace with np.nan
def to_float_or_nan(val):
    try:
        val = float(val)
    except ValueError:
        val = np.nan
    return val
combine_driving_test["tot_emp"] = combine_driving_test["tot_emp"].apply(to_float_or_nan)
combine_driving_test["h_median"] = combine_driving_test["h_median"].apply(to_float_or_nan)

In [47]:
#intermediate step - add column with toal paid out for that row
combine_driving_test["totalemp*wage"] = combine_driving_test[["tot_emp", "h_median"]].prod(axis=1, min_count=2)

In [84]:
#obtain a multiindex series including the median hourly pay for all CDL drivers weighted by the number of drivers in each employment category

total_pay = (combine_driving_test.groupby(["state", "year"])["totalemp*wage"].sum())/(combine_driving_test.groupby(["state", "year"])["tot_emp"].sum())

In [85]:
total_pay = total_pay.to_frame()
total_pay = total_pay.rename(columns={0 : "drivers_h_median"})

In [65]:
combine_driving_test.tail(10)

,state,occ_code,tot_emp,h_median,percent_of_median,year,totalemp*wage,sumtotempwage
4055,Wisconsin,driving,49090.0,24.09,NaN,2022,1182578.1,1182578.1
4056,Wisconsin,driving,21880.0,19.60,NaN,2022,428848.0,1611426.1
4057,Wisconsin,driving,7830.0,18.37,NaN,2022,143837.1,1755263.2
4058,Wisconsin,driving,2090.0,23.75,NaN,2022,49637.5,1804900.7
4059,Wisconsin,driving,3980.0,13.57,NaN,2022,54008.6,1858909.3
4061,Wyoming,driving,6020.0,23.93,NaN,2022,144058.6,144058.6
4062,Wyoming,driving,1880.0,17.67,NaN,2022,33219.6,177278.2
4063,Wyoming,driving,1480.0,18.75,NaN,2022,27750.0,205028.2
4064,Wyoming,driving,130.0,18.79,NaN,2022,2442.7,207470.9
4065,Wyoming,driving,410.0,12.28,NaN,2022,5034.8,212505.7


In [66]:
#check that my series is doing what I think it's doing... 
a2001 = (1290.0*8.40 + 8400.0*6.27 + 33270.0*14.37 + 14810.0*10.26)/(33270+14810+8400+1290)
w2022 = (6020*23.93 + 1880*17.67 + 1480*18.75 + 130*18.79 + 410*12.28)/(6020+1880+1480+130+410)

In [67]:
print(f"a2001 is {a2001}")
print(f"w2022 is {w2022}")

a2001 is 12.005270901852171
w2022 is 21.42194556451613


In [78]:
all_occ_df = full_df[full_df["occ_code"] == "all"][["state", "year", "h_median"]]
all_occ_df = all_occ_df.rename(columns={"h_median" : "all_occ_h_median"})
all_occ_df

,state,year,all_occ_h_median
0,Alabama,2001,11.02
5,Alaska,2001,16.12
9,Arizona,2001,12.33
14,Arkansas,2001,10.47
19,California,2001,14.31
...,...,...,...
4036,Virginia,2022,23.22
4042,Washington,2022,27.08
4048,West Virginia,2022,18.16
4054,Wisconsin,2022,21.95


In [86]:
combined_df = pd.merge(left=all_occ_df, right=total_pay, on=["state", "year"], how="outer")

In [91]:
combined_df = combined_df.round(2)
combined_df.to_csv("wage data")